Get foreground objects, create contours, find center of biggest contour and store into trajectory.

Can also visualize contour outlines and centers live here

In [ ]:
import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow

cap = cv.VideoCapture('vtest_cropped.mp4')
fgbg = cv.bgsegm.createBackgroundSubtractorMOG()

trajectory = []
while(1):
    ret, frame = cap.read()
    fgmask = fgbg.apply(frame)
    if not ret:  # video ended
        break

    contours, _ = cv.findContours(fgmask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    contours_filtered = []
    max_area, max_contour = 0, None

    for c in contours:
      area = cv.contourArea(c)
      if area > 500:  # ignore tiny blobs/noise
          contours_filtered.append(c)
          x, y, w, h = cv.boundingRect(c)
          center = (x + w//2, y + h//2)
          if area > max_area:
            max_area = area
            max_contour = center
          cv.circle(frame, center, 5, (0, 0, 255), -1)
    trajectory.append(max_contour)
    cv.circle(frame, max_contour, 10, (0, 255, 255), -1)

    cv.drawContours(frame, contours_filtered, -1, (0,255, 0), 5)

    if ret:
      # cv2_imshow(frame)
      print(len(trajectory))
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
cap.release()
cv.destroyAllWindows()
print(trajectory)



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


Attempt to smooth trajectory and amend faulty None values

In [ ]:
from scipy.signal import medfilt
trajectory = [None, None, (1233, 177), (1233, 177), None, (878, 545), (878, 545), (878, 545), (1267, 190), (1267, 190), (1267, 190), (816, 509), (816, 509), (822, 508), (838, 538), (838, 538), (838, 538), (927, 535), (927, 535), (927, 535), (932, 538), (932, 538), (937, 538), (940, 553), (940, 555), (940, 555), (952, 555), (952, 555), (952, 555), (955, 555), (959, 503), (959, 503), (968, 504), (968, 504), (968, 504), (970, 502), (970, 502), (970, 502), (972, 497), (979, 497), (979, 497), (979, 508), (979, 508), (979, 508), (976, 500), (976, 496), (976, 496), (972, 481), (976, 479), (976, 479), (919, 387), (919, 387), (918, 387), (911, 392), (911, 388), (911, 388), (911, 380), (911, 380), (911, 380), (909, 381), (909, 381), (899, 381), (897, 381), (897, 381), (897, 381), (905, 377), (905, 377), (905, 377), (905, 376), (905, 376), (905, 376), (899, 373), (899, 373), (897, 371), (879, 377), (879, 375), (879, 375), (876, 372), (875, 372), (875, 372), (869, 377), (869, 377), (866, 377), (859, 375), (859, 375), (859, 375), (848, 382), (848, 382), (848, 382), (843, 379), (843, 371), (843, 371), (842, 379), (842, 379), (841, 379), (841, 379), (841, 379), (841, 379), (841, 379), (841, 379), (841, 379), (843, 378), (843, 378), (843, 378), (1230, 74), (1230, 74), (1230, 74), (847, 368), (847, 368), (1220, 68), (1209, 62), (1209, 62), (1209, 62), (1192, 61), (1192, 61), (1192, 61), (1185, 61), (1185, 61), (1185, 61), (1182, 58), (1182, 58), (1182, 58), (1164, 49), (1164, 49), (1164, 49), (849, 357), (849, 357), (848, 357), (843, 339), (843, 339), (842, 339), (837, 339), (837, 339), (836, 339), (830, 349), (830, 349), (830, 349), (828, 348), (828, 348), (827, 348), (825, 348), (825, 348), (825, 348), (820, 347), (820, 347), (820, 347), (817, 343), (817, 343), (816, 343), (811, 357), (811, 357), (810, 357), (802, 360), (802, 360), (802, 360), (794, 360), (794, 360), (794, 360), (773, 355), (773, 355), (773, 355), (756, 353), (756, 353), (756, 353), (751, 344), (751, 344), (751, 344), (729, 346), (729, 346), (729, 346), (706, 344), (706, 344), (706, 344), (697, 344), (697, 344), (697, 344), (675, 333), (675, 333), (675, 333), (666, 329), (666, 329), (666, 329), (645, 317), (645, 317), (645, 317), (639, 321), (639, 321), (639, 321), (603, 321), (603, 321), (603, 321), (608, 320), (608, 320), (608, 320), (593, 316), (593, 316), (593, 316), (575, 308), (575, 308), (575, 308), (539, 305), (539, 305), (539, 305), (529, 304), (529, 304), (529, 304), (518, 305), (518, 305), (518, 305), (508, 304), (508, 304), (508, 304), (484, 295), (484, 295), (484, 295), (460, 285), (460, 285), (460, 285), (440, 279), (440, 279), (440, 279), (414, 281), (414, 281), (414, 281), (404, 281), (404, 281), (404, 281), (390, 274), (390, 275), (390, 275), (354, 262), (354, 262), (354, 262), (347, 258), (347, 258), (347, 258), (345, 257), (345, 257), (345, 257), (342, 257), (342, 257), (342, 257), (335, 242), (335, 242), (335, 242), (328, 230), (328, 230), (328, 230), (324, 229), (324, 229), (324, 229), (325, 219), (325, 219), (325, 219), (321, 215), (321, 215), (321, 215), (317, 204), (317, 204), (317, 204), (312, 206), (312, 206), (312, 206), (305, 194), (305, 194), (305, 194), (302, 191), (302, 191), (302, 191), (301, 189), (301, 189), (301, 189), (297, 168), (297, 168), (297, 168), (289, 168), (289, 168), (289, 168), (285, 163), (285, 163), (285, 163), (281, 160), (281, 160), (281, 160), (267, 159), (267, 159), (267, 159), (261, 141), (261, 141), (261, 141), (256, 140), (256, 140), (256, 140), (244, 133), (244, 133), (244, 133), (239, 126), (239, 126), (239, 126), (233, 115), (233, 115), (233, 115), (223, 117), (223, 117), (223, 117), (214, 109), (214, 109), (214, 109), (204, 105), (204, 105), (204, 105), (198, 109), (198, 109), (198, 109), (177, 97), (177, 97), (177, 97), (172, 97), (172, 97), (172, 97), (169, 96), (169, 96), (169, 96), (168, 93), (168, 93), (168, 93), (163, 90), (163, 90), (163, 90), (164, 84), (164, 84), (164, 84), (165, 83), (165, 83), (165, 83), (167, 76), (166, 76), (166, 76), (168, 75), (168, 75), (168, 75), (173, 70), (173, 70), (173, 70), (181, 67), (181, 67), (181, 67), (185, 67), (185, 67), (185, 67), (192, 63), (192, 63), (192, 63), (210, 60), (210, 60), (210, 60), (215, 59), (215, 59), (215, 59), (218, 58), (218, 58), (218, 58), (224, 54), (224, 54), (224, 54), (1244, 36), (1244, 36), (1244, 36)]

for i, loc in enumerate(trajectory):
  if not loc:

    before = None
    for temp in range(i, len(trajectory)):
      if trajectory[temp]:
        before = trajectory[temp]
        break

    after = None
    for temp in range(0, i, -1):
      if trajectory[temp]:
        after = trajectory[temp]
        break

    if before and after:
      trajectory[i] = (before + after) / 2
    elif before:
      trajectory[i] = before
    elif after:
      trajectory[i] = after

traj = np.array(trajectory)
x_smooth = medfilt(traj[:,0], kernel_size=5)
y_smooth = medfilt(traj[:,1], kernel_size=5)
smoothed_traj = np.stack([x_smooth, y_smooth], axis=1)

368
[(1233, 177), (1233, 177), (1233, 177), (1233, 177), (878, 545), (878, 545), (878, 545), (878, 545), (1267, 190), (1267, 190), (1267, 190), (816, 509), (816, 509), (822, 508), (838, 538), (838, 538), (838, 538), (927, 535), (927, 535), (927, 535), (932, 538), (932, 538), (937, 538), (940, 553), (940, 555), (940, 555), (952, 555), (952, 555), (952, 555), (955, 555), (959, 503), (959, 503), (968, 504), (968, 504), (968, 504), (970, 502), (970, 502), (970, 502), (972, 497), (979, 497), (979, 497), (979, 508), (979, 508), (979, 508), (976, 500), (976, 496), (976, 496), (972, 481), (976, 479), (976, 479), (919, 387), (919, 387), (918, 387), (911, 392), (911, 388), (911, 388), (911, 380), (911, 380), (911, 380), (909, 381), (909, 381), (899, 381), (897, 381), (897, 381), (897, 381), (905, 377), (905, 377), (905, 377), (905, 376), (905, 376), (905, 376), (899, 373), (899, 373), (897, 371), (879, 377), (879, 375), (879, 375), (876, 372), (875, 372), (875, 372), (869, 377), (869, 377), (866

Output video based on trajectory

In [ ]:

cap = cv.VideoCapture('vtest_cropped.mp4')

height, width = int(cap.get(4)), int(cap.get(3))
out = cv.VideoWriter('output.mp4', cv.VideoWriter_fourcc(*'mp4v'), 30, (width, height))

for loc in smoothed_traj:
    ret, frame = cap.read()
    cv.circle(frame, loc, 5, (0, 0, 255), -1)

    if not ret:
        break
    if k == 27:
        break
    out.write(frame)

out.release()
